In [50]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
from collections import namedtuple

from vega_sim.null_service import VegaServiceNull

import numpy as np
import time

In [52]:
WalletConfig = namedtuple("WalletConfig", ["name", "passphrase"])

MM_WALLET = WalletConfig("mm", "pin")

TRADER_1_WALLET = WalletConfig("T1", "pin")

TRADER_2_WALLET = WalletConfig("T2", "pin")

TERMINATE_WALLET = WalletConfig("Settle", "pin")

wallets = [MM_WALLET, TRADER_1_WALLET, TRADER_2_WALLET, TERMINATE_WALLET]

In [53]:
vega = VegaServiceNull(run_with_console=True, block_duration='1s')
vega.start()

/var/folders/67/mjxp58z56yj83_1x0372gkwr0000gn/T/tmpisr0fafw


In [54]:
print(vega.wallet_url)

http://localhost:52858


#### Create the wallets

In [55]:
for wallet in wallets:
    vega.create_wallet(wallet.name, passphrase=wallet.passphrase)
vega.forward('10s')

#### Create the asset + market

In [56]:
vega.mint(
    MM_WALLET.name,
    asset="VOTE",
    amount=10000,
)

vega.forward("20s")
vega.create_asset(
    MM_WALLET.name,
    name="tDAI",
    symbol="tDAI",
    decimals=5,
    max_faucet_amount=10e10,
)

In [57]:
tdai_id = vega.find_asset_id(symbol="tDAI")

In [58]:
vega.mint(
    MM_WALLET.name,
    tdai_id,
    amount=10e5
)
vega.mint(
    TRADER_1_WALLET.name,
    tdai_id,
    amount=10e5
)
vega.mint(
    TRADER_2_WALLET.name,
    tdai_id,
    amount=10e5
)

#### Create the market and set up liquidity

In [59]:
vega.create_simple_market(
        market_name="BTC:DAI_Mar22",
        proposal_wallet=MM_WALLET.name,
        settlement_asset_id=tdai_id,
        termination_wallet=TERMINATE_WALLET.name,
        position_decimals=2,
        market_decimals=3,
    )

In [60]:
market_id = vega.all_markets()[0].id

Using function with raw data from data-node VegaService.all_markets. Be wary if prices/positions are not converted from int form


In [61]:
vega.submit_simple_liquidity(
    wallet_name=MM_WALLET.name,
    market_id=market_id,
    commitment_amount=3e5,
    fee=0.005,
    reference_buy="PEGGED_REFERENCE_BEST_BID",
    reference_sell="PEGGED_REFERENCE_BEST_ASK",
    delta_buy=10,
    delta_sell=10,
    is_amendment=True,
)

In [62]:
# Submit orders which will stay on the book and create best bid / ask 
vega.submit_order(
    trading_wallet=MM_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=1,
    price=95,
    wait=False,
)

vega.submit_order(
    trading_wallet=MM_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=1,
    price=105,
    wait=False,
)



In [63]:
for wallet in wallets:
    general, margin, bond = vega.party_account(wallet_name=wallet.name, asset_id=tdai_id, market_id=market_id)
    print(wallet.name + " General = "+str(general))
    print(wallet.name + " Margin  = "+str(margin))
    print(wallet.name + " Bond    = "+str(bond))

mm General = 699986.33776
mm Margin  = 13.66224
mm Bond    = 300000.0
T1 General = 1000000.0
T1 Margin  = nan
T1 Bond    = nan
T2 General = 1000000.0
T2 Margin  = nan
T2 Bond    = nan
Settle General = nan
Settle Margin  = nan
Settle Bond    = nan


#### Do some trading

In [64]:
# Do a trade which will cross
vega.submit_order(
    trading_wallet=TRADER_1_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=1,
    price=100,
    wait=False,
)

vega.submit_order(
    trading_wallet=TRADER_2_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=1,
    price=100,
    wait=False,
)
vega.forward('10s')

# We will now have mark price of 100 


In [65]:
for wallet in wallets:
    general, margin, bond = vega.party_account(wallet_name=wallet.name, asset_id=tdai_id, market_id=market_id)
    print(wallet.name + " General = "+str(general))
    print(wallet.name + " Margin  = "+str(margin))
    print(wallet.name + " Bond    = "+str(bond))

mm General = 999993.49418
mm Margin  = 6.50582
mm Bond    = 0.0
T1 General = 999987.63244
T1 Margin  = 12.36756
T1 Bond    = nan
T2 General = 999986.98836
T2 Margin  = 13.01164
T2 Bond    = nan
Settle General = nan
Settle Margin  = nan
Settle Bond    = nan


In [66]:
vega.settle_market(settlement_wallet = TERMINATE_WALLET.name, settlement_price = 95, market_id=market_id)

In [69]:
vega.forward('10s')

for wallet in wallets:
    general, margin, bond = vega.party_account(wallet_name=wallet.name, asset_id=tdai_id, market_id=market_id)
    print(wallet.name + " General = "+str(general))
    print(wallet.name + " Margin  = "+str(margin))
    print(wallet.name + " Bond    = "+str(bond))

    positions = vega.positions_by_market(wallet_name=wallet.name, market_id=market_id)
    print(positions)


Using function with raw data from data-node VegaService.positions_by_market. Be wary if prices/positions are not converted from int form
Using function with raw data from data-node VegaService.positions_by_market. Be wary if prices/positions are not converted from int form
Using function with raw data from data-node VegaService.positions_by_market. Be wary if prices/positions are not converted from int form
Using function with raw data from data-node VegaService.positions_by_market. Be wary if prices/positions are not converted from int form


mm General = 999993.49418
mm Margin  = 6.50582
mm Bond    = 0.0
[market_id: "3b7adda63cfe46ac362452a69f4b7b7f9ff6f2c03b89535b8b97f5d19355ced5"
party_id: "46872e78ad5b59532d1c7dcdd120b87c098beeccb4aac94052d957e0939c84ad"
realised_pnl: "0"
unrealised_pnl: "0"
average_entry_price: "0"
]
T1 General = 999987.63244
T1 Margin  = 12.36756
T1 Bond    = nan
[market_id: "3b7adda63cfe46ac362452a69f4b7b7f9ff6f2c03b89535b8b97f5d19355ced5"
party_id: "a52dffd52ed886a91f4d8b60a070f70c0ecbbbf6ca8fcf3872f8783a28419b12"
open_volume: 100
realised_pnl: "0"
unrealised_pnl: "0"
average_entry_price: "10000000"
]
T2 General = 999986.98836
T2 Margin  = 13.01164
T2 Bond    = nan
[market_id: "3b7adda63cfe46ac362452a69f4b7b7f9ff6f2c03b89535b8b97f5d19355ced5"
party_id: "09f6ab18d3d4a5af19855fdabbb0b1e2d0eb36150a1094ee1fbc4376fdd37ec8"
open_volume: -100
realised_pnl: "0"
unrealised_pnl: "0"
average_entry_price: "10000000"
]
Settle General = nan
Settle Margin  = nan
Settle Bond    = nan
[]


In [49]:
vega.stop()